In [27]:


# STEP 1: Import Required Libraries
import pandas as pd
import numpy as np

# STEP 2: Load Data
df_all = pd.read_csv("Final_Irrigation_Dispatch_Model_2Pivots.csv")

#  STEP 3: Define Parameters
battery_capacity = 50         # kWh
initial_soc = 40               # Starting SOC
min_soc = 10                   # Minimum allowed
max_charge_rate = 48          # kWh per hour
max_discharge_rate = 15       # kWh per hour
n_hours_per_day = 24
n_days = len(df_all) // n_hours_per_day

# STEP 4: Rule-Based Dispatch Function
def dispatch_day(df_day, soc_start):
    n = len(df_day)
    soc = soc_start
    soc_list = []
    charge_list = []
    discharge_list = []
    grid_list = []
    cost_list = []

    for i in range(n):
        load = df_day.loc[df_day.index[i], 'Irrigation_Load_kWh']
        solar = df_day.loc[df_day.index[i], 'Solar_Generation_kWh']
        price = df_day.loc[df_day.index[i], 'Grid_Price_ZMW_per_kWh']

        # Charge battery with available solar if not full
        charge = min(max_charge_rate, max(0, solar), battery_capacity - soc)
        soc += charge

        # Use solar for load first
        remaining_load = load - (solar - charge)

        # Discharge battery if needed
        discharge = min(max_discharge_rate, soc, max(0, remaining_load))
        soc -= discharge
        remaining_load -= discharge

        # Grid import covers the rest
        grid = max(0, remaining_load)
        cost = grid * price

        # Record
        soc_list.append(soc)
        charge_list.append(charge)
        discharge_list.append(discharge)
        grid_list.append(grid)
        cost_list.append(cost)

    df_day['Battery_Charge_kWh'] = charge_list
    df_day['Battery_Discharge_kWh'] = discharge_list
    df_day['Grid_Import_kWh'] = grid_list
    df_day['SOC_kWh'] = soc_list
    df_day['Cost_ZMW'] = cost_list

    return df_day, soc

#  STEP 5: Apply Rule-Based Dispatch Per Day
results = []
soc = initial_soc

for day in range(n_days):
    print(f"\u23f3 Running dispatch for Day {day+1}")
    start = day * n_hours_per_day
    end = start + n_hours_per_day
    df_day = df_all.iloc[start:end].copy()

    df_day_out, soc = dispatch_day(df_day, soc)
    results.append(df_day_out)

# STEP 6: Save Results
if results:
    df_final = pd.concat(results, ignore_index=True)
    df_final.to_excel("Rule_Based_Dispatch_50kWh.xlsx", index=False)
    print("\u2705 Saved: Rule_Based_Dispatch_50kWh.xlsx")
else:
    print(" No results saved.")


⏳ Running dispatch for Day 1
⏳ Running dispatch for Day 2
⏳ Running dispatch for Day 3
⏳ Running dispatch for Day 4
⏳ Running dispatch for Day 5
⏳ Running dispatch for Day 6
⏳ Running dispatch for Day 7
⏳ Running dispatch for Day 8
⏳ Running dispatch for Day 9
⏳ Running dispatch for Day 10
⏳ Running dispatch for Day 11
⏳ Running dispatch for Day 12
⏳ Running dispatch for Day 13
⏳ Running dispatch for Day 14
⏳ Running dispatch for Day 15
⏳ Running dispatch for Day 16
⏳ Running dispatch for Day 17
⏳ Running dispatch for Day 18
⏳ Running dispatch for Day 19
⏳ Running dispatch for Day 20
⏳ Running dispatch for Day 21
⏳ Running dispatch for Day 22
⏳ Running dispatch for Day 23
⏳ Running dispatch for Day 24
⏳ Running dispatch for Day 25
⏳ Running dispatch for Day 26
⏳ Running dispatch for Day 27
⏳ Running dispatch for Day 28
⏳ Running dispatch for Day 29
⏳ Running dispatch for Day 30
✅ Saved: Rule_Based_Dispatch_50kWh.xlsx
